<a href="https://colab.research.google.com/github/rahiakela/automl-experiments/blob/main/auto-keras-practice-works/06_structured_data_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Structured Data Regression

Reference:

https://autokeras.com/tutorial/structured_data_regression/

##Setup

In [1]:
!pip -q install autokeras

     |████████████████████████████████| 166 kB 4.1 MB/s 
     |████████████████████████████████| 454.4 MB 23 kB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 462 kB 27.3 MB/s 
     |████████████████████████████████| 4.0 MB 28.1 MB/s 
     |████████████████████████████████| 1.2 MB 25.9 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

import autokeras as ak

##A Simple Example

The first step is to prepare your data. Here we use the [California housing
dataset](
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)
as an example.


In [3]:
house_dataset = fetch_california_housing()

In [5]:
df = pd.DataFrame(np.concatenate((house_dataset.data, house_dataset.target.reshape(-1, 1)), axis=1), columns=house_dataset.feature_names + ["Price"])

train_size = int(df.shape[0] * 0.9)
df[: train_size].to_csv("train.csv", index=False)
df[: train_size].to_csv("eval.csv", index=False)

train_file_path = "train.csv"
test_file_path = "eval.csv"

The second step is to run the
[StructuredDataRegressor](/structured_data_regressor).
As a quick demo, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [7]:
# Initialize the structured data regressor
reg = ak.StructuredDataRegressor(overwrite=True, max_trials=3)

# Feed the structured data regressor with training data.
reg.fit(
    train_file_path,  # The path to the train.csv file
    "Price",          #  The name of the label column
    epochs=10
)

Trial 3 Complete [00h 00m 21s]
val_loss: 0.9433661103248596

Best val_loss So Far: 0.9002568125724792
Total elapsed time: 00h 01m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.2853 - mean_squared_error: 1.2853
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6597 - mean_squared_error: 0.6597
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5735 - mean_squared_error: 0.5735
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5381 - mean_squared_error: 0.5381
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5236 - mean_squared_error: 0.5236
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5171 - mean_squared_error: 0.5171
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5085 - mean_squared_error: 0.5085
Epoch 8/10
581/581 [==============================] - 1s 2

In [8]:
# Predict with the best model
predicted_y = reg.predict(test_file_path)
# Evaluate the best model with testing data
print(reg.evaluate(test_file_path, "Price"))

581/581 [==============================] - 1s 2ms/step - loss: 0.5979 - mean_squared_error: 0.5979
[0.5979295372962952, 0.5979295372962952]


##Data Format

The AutoKeras StructuredDataRegressor is quite flexible for the data format.

The example above shows how to use the CSV files directly. Besides CSV files,
it also supports numpy.ndarray, pandas.DataFrame or [tf.data.Dataset](
https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=stable). The
data should be two-dimensional with numerical or categorical values.

For the regression targets, it should be a vector of numerical values.
AutoKeras accepts `numpy.ndarray`, `pandas.DataFrame`, or `pandas.Series`.

The following examples show how the data can be prepared with `numpy.ndarray`,
`pandas.DataFrame`, and `tensorflow.data.Dataset`.


In [9]:
# x_train as pandas.DataFrame, y_train as pandas.Series
x_train = pd.read_csv(train_file_path)
print(type(x_train))  # pandas.DataFrame
y_train = x_train.pop("Price")
print(type(y_train))  # pandas.Series

# You can also use pandas.DataFrame for y_train.
y_train = pd.DataFrame(y_train)
print(type(y_train))  # pandas.DataFrame

# You can also use numpy.ndarray for x_train and y_train.
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(type(x_train))  # numpy.ndarray
print(type(y_train))  # numpy.ndarray

# Preparing testing data.
x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("Price")

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [10]:
# It tries 10 different models.
reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the structured data regressor with training data.
reg.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(x_test)
# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))

Trial 3 Complete [00h 00m 19s]
val_loss: 0.9210050106048584

Best val_loss So Far: 0.8640338182449341
Total elapsed time: 00h 00m 51s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.1592 - mean_squared_error: 1.1592
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6278 - mean_squared_error: 0.6278
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5558 - mean_squared_error: 0.5558
Epoch 4/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5288 - mean_squared_error: 0.5288
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5125 - mean_squared_error: 0.5125
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5034 - mean_squared_error: 0.5034
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.4960 - mean_squared_error: 0.4960
Epoch 8/10
581/581 [==============================] - 1s 2

The following code shows how to convert `numpy.ndarray` to `tf.data.Dataset`.



In [11]:
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))

reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(test_set)
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))

Trial 3 Complete [00h 00m 26s]
val_loss: 1.1470242738723755

Best val_loss So Far: 0.8675209879875183
Total elapsed time: 00h 01m 23s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 4s 5ms/step - loss: 1.2792 - mean_squared_error: 1.2792
Epoch 2/10
581/581 [==============================] - 3s 4ms/step - loss: 0.6216 - mean_squared_error: 0.6216
Epoch 3/10
581/581 [==============================] - 3s 6ms/step - loss: 0.5517 - mean_squared_error: 0.5517
Epoch 4/10
581/581 [==============================] - 3s 4ms/step - loss: 0.5358 - mean_squared_error: 0.5358
Epoch 5/10
581/581 [==============================] - 2s 4ms/step - loss: 0.5250 - mean_squared_error: 0.5250
Epoch 6/10
581/581 [==============================] - 3s 5ms/step - loss: 0.5162 - mean_squared_error: 0.5162
Epoch 7/10
581/581 [==============================] - 3s 4ms/step - loss: 0.5105 - mean_squared_error: 0.5105
Epoch 8/10
581/581 [==============================] - 2s 4

You can also specify the column names and types for the data as follows.  The
`column_names` is optional if the training data already have the column names,
e.g.  `pandas.DataFrame`, CSV file.  Any column, whose type is not specified will
be inferred from the training data.


In [14]:
# Initialize the structured data regressor
reg2 = ak.StructuredDataRegressor(
    column_names=["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"],
    column_types={"MedInc": "numerical", "Latitude": "numerical"},
    max_trials=10,
    overwrite=True
)

# Feed the tensorflow Dataset to the regressor.
reg2.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = reg2.predict(test_set)
# Evaluate the best model with testing data.
print(reg2.evaluate(test_set))

Trial 10 Complete [00h 00m 39s]
val_loss: 1.3603469133377075

Best val_loss So Far: 1.168041467666626
Total elapsed time: 00h 06m 56s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 3s 5ms/step - loss: 1.0392 - mean_squared_error: 1.0392
Epoch 2/10
581/581 [==============================] - 3s 5ms/step - loss: 0.7178 - mean_squared_error: 0.7178
Epoch 3/10
581/581 [==============================] - 4s 7ms/step - loss: 0.6466 - mean_squared_error: 0.6466
Epoch 4/10
581/581 [==============================] - 2s 4ms/step - loss: 0.5919 - mean_squared_error: 0.5919
Epoch 5/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5930 - mean_squared_error: 0.5930
Epoch 6/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5691 - mean_squared_error: 0.5691
Epoch 7/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5694 - mean_squared_error: 0.5694
Epoch 8/10
581/581 [==============================] - 2s 4

#### Validation Data

By default, AutoKeras use the last 20% of training data as validation data.  As
shown in the example below, you can use `validation_split` to specify the
percentage.


In [ ]:
reg2.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)

# Predict with the best model.
predicted_y = reg2.predict(test_set)
# Evaluate the best model with testing data.
print(reg2.evaluate(test_set))

You can also use your own validation set
instead of splitting it from the training data with `validation_data`.

In [ ]:
split = 500
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]

reg2.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)

# Predict with the best model.
predicted_y = reg2.predict(test_set)
# Evaluate the best model with testing data.
print(reg2.evaluate(test_set))

## Customized Search Space

For advanced users, you may customize your search space by using
[AutoModel](/auto_model/#automodel-class) instead of
[StructuredDataRegressor](/structured_data_regressor). You can configure the
[StructuredDataBlock](/block/#structureddatablock-class) for some high-level
configurations, e.g., `categorical_encoding` for whether to use the
[CategoricalToNumerical](/block/#categoricaltonumerical-class). 

You can also do
not specify these arguments, which would leave the different choices to be
tuned automatically. 

See the following example for detail.

In [15]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)

reg3 = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=3)
reg3.fit(x_train, y_train, epochs=10)

Trial 3 Complete [00h 00m 21s]
val_loss: 1.6993786096572876

Best val_loss So Far: 1.6993786096572876
Total elapsed time: 00h 00m 57s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.4211 - mean_squared_error: 1.4211
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3807 - mean_squared_error: 1.3807
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3721 - mean_squared_error: 1.3721
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3818 - mean_squared_error: 1.3818
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3810 - mean_squared_error: 1.3810
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3810 - mean_squared_error: 1.3810
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 1.3810 - mean_squared_error: 1.3810
Epoch 8/10
581/581 [==============================] - 1s 2

The usage of [AutoModel](/auto_model/#automodel-class) is similar to the
[functional API](https://www.tensorflow.org/guide/keras/functional) of Keras.
Basically, you are building a graph, whose edges are blocks and the nodes are
intermediate outputs of blocks.  To add an edge from `input_node` to
`output_node` with `output_node = ak.[some_block]([block_args])(input_node)`.

You can even also use more fine grained blocks to customize the search space
even further. See the following example.


In [16]:
input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.RegressionHead()(output_node)

reg4 = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=3)
reg4.fit(x_train, y_train, epochs=10)

Trial 3 Complete [00h 00m 18s]
val_loss: 2.652089834213257

Best val_loss So Far: 2.54856276512146
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 2.8250 - mean_squared_error: 2.8250
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 1.9858 - mean_squared_error: 1.9858
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 1.7852 - mean_squared_error: 1.7852
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 1.7009 - mean_squared_error: 1.7009
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 1.6353 - mean_squared_error: 1.6353
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 1.6069 - mean_squared_error: 1.6069
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 1.5705 - mean_squared_error: 1.5705
Epoch 8/10
581/581 [==============================] - 1s 2ms/

You can also export the best model found by AutoKeras as a Keras Model.

In [17]:
model = reg4.export_model()
model.summary()
# numpy array in object (mixed type) is not supported.
# you need convert it to unicode or float first.
model.predict(x_train)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 32)                288       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056  

array([[2.444552 ],
       [2.194388 ],
       [2.3857846],
       ...,
       [1.7851417],
       [1.9181583],
       [1.9060069]], dtype=float32)

## Reference

[StructuredDataRegressor](/structured_data_regressor),

[AutoModel](/auto_model/#automodel-class),

[StructuredDataBlock](/block/#structureddatablock-class),

[DenseBlock](/block/#denseblock-class),

[StructuredDataInput](/node/#structureddatainput-class),

[RegressionHead](/block/#regressionhead-class),

[CategoricalToNumerical](/block/#categoricaltonumerical-class).